In [1]:
from dataclasses import dataclass

import numpy as np
import matplotlib.pyplot as plt

In [2]:
@dataclass
class Configuration:
    n_paths: int
    n_steps: int

In [3]:
@dataclass
class EuropeanOption:
    stock_price: float
    strike_price: float
    risk_free_rate: float
    volatility: float
    time_to_maturity: float
        
    def disc_fact(self):
        return np.exp(-1.0 * self.risk_free_rate * self.time_to_maturity)
    
    def payoff(self, observations):
        raise NotImplementedError

    def pv(self, observations):
        return self.disc_fact() * self.payoff(observations)
    
@dataclass
class EuropeanCall(EuropeanOption):
    def payoff(self, fixing):
        return max(fixing - self.strike_price, 0)
    
@dataclass
class EuropeanPut(EuropeanOption):
    def payoff(self, fixing):
        return max(self.strike_price - fixing, 0)
    
@dataclass
class EuropeanCallDigitalCash(EuropeanOption):
    def payoff(self, fixing):
        return 1 if fixing > self.strike_price else 0
    
@dataclass
class EuropeanPutDigitalCash(EuropeanOption):
    def payoff(self, fixing):
        return 1 if fixing < self.strike_price else 0

In [4]:
class GBMModel:
    def __init__(self, configuration):
        self.configuration = configuration
        
    # simulate risk factors using GBM stochastic differential equation
    def simulate(self, security):
        prices = []
        # for this example, we only are concerned with one time step as it’s an European option
        dt = 1
        for path in range(self.configuration.n_paths):
            normal_random_number = np.random.normal(0, 1)           
            drift = (security.risk_free_rate - 0.5 * (security.volatility ** 2)) * dt
            diffusion = security.volatility * np.sqrt(dt) * normal_random_number
            price = security.stock_price * np.exp(drift + diffusion)
            prices.append(price)    
        return prices

In [5]:
class PayoffPricer:
    def calculate_price(self, security, simulation_results):
        pvs = 0
        total_paths = len(simulation_results)
        for simulation_result in simulation_results:
            pvs += security.pv(simulation_result)
        return pvs / total_paths

In [6]:
class MonteCarloPricer:
    def __init__(self, configuration, model):
        """ Instantiate with a configuration and the model """
        self.configuration = configuration
        self.model = model
        
    def price(self, security, payoff_pricer):
        """ simulate trade and calculate price """
        simulation_results = self.model.simulate(security)
        price = payoff_pricer.calculate_price(security, simulation_results)
        return price

In [7]:
configuration = Configuration(100000, 8)
security = EuropeanCallDigitalCash(100, 100, 0.05, 0.2, 1)
model = GBMModel(configuration)
payoff_pricer = PayoffPricer()
pricer = MonteCarloPricer(configuration, model)
pv = pricer.price(security, payoff_pricer)
print(pv)

0.5326789654257794


References:
1. https://realpython.com/python-data-classes/#inheritance
2. https://medium.com/fintechexplained/monte-carlo-simulation-engine-in-python-a1fa5043c613